# TF on GKE

This notebook shows how to train [TensorFlow Inception](https://github.com/tensorflow/models/tree/master/research/slim) on GKE using [TfJobs](https://github.com/tensorflow/k8s)

## Requirements

To run this notebook you must have the following installed
  * gcloud
  * kubectl
  * helm
  * kubernetes python client library
  
There is a Docker image based on Datalab suitable for running this notebook.

You can start that container as follows

```
docker run --name=gke-datalab -p "127.0.0.1:8081:8080" \
    -v "${HOME}:/content/datalab/home" \
    -v /var/run/docker.sock:/var/run/docker.sock -d  -e "PROJECT_ID=" \
    gcr.io/tf-on-k8s-dogfood/gke-datalab:v20171103-73616f0
```
  * You need to map in docker if you want tobuild docker images inside the container.
  * Alternatively, you can set "use_gcb" to true in order to build the images using Google Container Builder
  
Additionally the [py package](https://github.com/tensorflow/k8s/tree/master/py) must be a top level package importable as py
  * If you cloned [tensorflow/k8s](https://github.com/tensorflow/k8s) and are running this notebook in place the path with be configured automatically

## Preliminaries

In [2]:
# Turn on autoreloading
%load_ext autoreload
%autoreload 2

import a bunch of modules and set some constants

In [1]:
from __future__ import print_function

import logging
import os
import sys

# Assumes we are running inside the cloned repo.
# Try to setup the path so we can import py as a top level package
ROOT_DIR = os.path.abspath(os.path.join("../.."))
if os.path.exists(os.path.join(ROOT_DIR, "py")):
  if not ROOT_DIR in sys.path:
    sys.path.append(ROOT_DIR)
  
import kubernetes
from kubernetes import client as k8s_client
from kubernetes import config as k8s_config
from kubernetes.client.rest import ApiException
from kubernetes.client.models.v1_label_selector import V1LabelSelector
import datetime
from googleapiclient import discovery
from googleapiclient import errors
from oauth2client.client import GoogleCredentials
from pprint import pprint
try:
  from py import build_and_push_image
  from py import util
except ImportError:
  raise ImportError("Please ensure the py package in https://github.com/tensorflow/k8s is a top level package")
import StringIO
import subprocess
import urllib
import urllib2
import time
import yaml

logging.getLogger().setLevel(logging.INFO)

TF_JOB_GROUP = "tensorflow.org"
TF_JOB_VERSION = "v1alpha1"
TF_JOB_PLURAL = "tfjobs"
TF_JOB_KIND = "TfJob"


### Configure the notebook for your use
Change the constants defined below.
  1. Change **project** to a project you have access to.
     * GKE should be enabled for that project
  1. Change **data_dir** and **job_dir**
     * Use a GCS bucket that you have access to
     * Ensure the service account on your GKE cluster can read/write to this GCS bucket

* Optional change the cluster name

In [9]:
project="cloud-ml-dev"
zone="us-east1-d"
cluster_name="gke-tf-example"
registry = "gcr.io/" + project
dataset_name = "flowers"
data_dir = os.path.join("gs://cloud-ml-dev_jlewi/inception/data", dataset_name)
job_dirs = "gs://cloud-ml-dev_jlewi/inception/jobs"
gke = discovery.build("container", "v1")
namespace = "default"

# Whether to build containers using Google Container Builder.
# Set to false it will build by shelling out to docker build.
use_gcb = "false"

## GKE Cluster Setup

* The instructions below create a **CPU** cluster
* To create a GKE cluster with GPUs sign up for the [GKE GPU Alpha](https://goo.gl/forms/ef7eh2x00hV3hahx1)
* To use GPUs set accelerator and accelerator_count
* For a full list of cluster options see the [Cluster object](https://cloud.google.com/container-engine/reference/rest/v1/projects.zones.clusters#Cluster) 
  in the GKE API docs

To use an existing GKE cluster call **configure_kubectl** but not **create_cluster**

* The code below issues a GKE request to create the cluster by calling util.create_cluster
  * util.create_cluster uses the GKE python client library
* After creating the cluster we call util.configure_kubectl
  * This configures your machine to talk to the K8s master of the newly created cluster

In [22]:
reload(util)
machine_type = "n1-standard-8"
use_gpu = True
if use_gpu:
  accelerator = "nvidia-tesla-k80"
  accelerator_count = 1
else:
  accelerator = None
  accelerator_count = 0

cluster_request = {
    "cluster": {
        "name": cluster_name,
        "description": "A GKE cluster for TF.",
        "initialNodeCount": 1,
        "nodeConfig": {
            "machineType": machine_type,
            "oauthScopes": [
              "https://www.googleapis.com/auth/cloud-platform",
            ],
        },
        # TODO(jlewi): Stop pinning GKE version once 1.8 becomes the default. 
        "initialClusterVersion": "1.8.1-gke.1",
    }
}

if bool(accelerator) != (accelerator_count > 0):
    raise ValueError("If accelerator is set accelerator_count must be  > 0")
    
if accelerator:
  # TODO(jlewi): Stop enabling Alpha once GPUs make it out of Alpha
  cluster_request["cluster"]["enableKubernetesAlpha"] = True

  cluster_request["cluster"]["nodeConfig"]["accelerators"] = [
      {
        "acceleratorCount": accelerator_count,
        "acceleratorType": accelerator,
      },
  ]
util.create_cluster(gke, project, zone, cluster_request)

util.configure_kubectl(project, zone, cluster_name)

k8s_config.load_kube_config()

# Create an API client object to talk to the K8s master.
api_client = k8s_client.ApiClient()

INFO:googleapiclient.discovery:URL being requested: POST https://container.googleapis.com/v1/projects/cloud-ml-dev/zones/us-east1-d/clusters?alt=json
INFO:root:Creating cluster; project=cloud-ml-dev, zone=us-east1-d, name=gke-tf-example
ERROR:root:Exception occured creating cluster: <HttpError 409 when requesting https://container.googleapis.com/v1/projects/cloud-ml-dev/zones/us-east1-d/clusters?alt=json returned "The resource "projects/cloud-ml-dev/zones/us-east1-d/clusters/gke-tf-example" already exists.">, status: 409
INFO:root:Configuring kubectl
INFO:root:Running: gcloud --project=cloud-ml-dev container clusters --zone=us-east1-d get-credentials gke-tf-example 
cwd=None
INFO:root:Subprocess output:
Fetching cluster endpoint and auth data.
kubeconfig entry generated for gke-tf-example.

INFO:requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): accounts.google.com


### Install the Operator

* We need to deploy the [TfJob](https://github.com/tensorflow/k8s) custom resource on our K8s cluster
* TfJob is deployed using the [helm](https://github.com/kubernetes/helm) package manager so first we need to setup helm on our cluster

In [98]:
util.setup_cluster(api_client)

INFO:root:Creating service account for tiller.
INFO:root:Service account tiller already exists.
INFO:root:Role binding for service account tiller already exists.
INFO:root:Running: helm init --service-account=tiller 
cwd=None
INFO:root:Subprocess output:
$HELM_HOME has been configured at /root/.helm.
(Use --client-only to suppress this message, or --upgrade to upgrade Tiller to the current version.)
Happy Helming!

INFO:root:GPUs detected in cluster.
INFO:root:Install GPU Drivers.
INFO:root:GPU driver daemon set has already been installed
INFO:root:tiller is ready
INFO:root:GPUs are available.


Now that helm is setup we can deploy the TfJob CRD

In [112]:
CHART="https://storage.googleapis.com/tf-on-k8s-dogfood-releases/latest/tf-job-operator-chart-latest.tgz"
util.run(["helm", "install", CHART, "-n", "tf-job", "--wait", "--replace", "--set", "rbac.install=true,cloud=gke"])

INFO:root:Running: helm install https://storage.googleapis.com/tf-on-k8s-dogfood-releases/latest/tf-job-operator-chart-latest.tgz -n tf-job --wait --replace --set rbac.install=true,cloud=gke 
cwd=None
INFO:root:Subprocess output:
NAME:   tf-job
LAST DEPLOYED: Fri Nov  3 02:00:48 2017
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Pod(related)
NAME                             READY  STATUS   RESTARTS  AGE
tf-job-operator-b4598cf8c-fkbc2  1/1    Running  0         2s

==> v1/ConfigMap
NAME                    DATA  AGE
tf-job-operator-config  1     2s

==> v1/ServiceAccount
NAME             SECRETS  AGE
tf-job-operator  1        2s

==> v1beta1/ClusterRole
NAME             AGE
tf-job-operator  2s

==> v1beta1/ClusterRoleBinding
NAME             AGE
tf-job-operator  2s

==> v1beta1/Deployment
NAME             DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
tf-job-operator  1        1        1           1          2s





## Build Docker images

To run a TensorFlow program on K8s we need to package our code as Docker images.

The [Dockerfile](https://github.com/jlewi/k8s/blob/73616f09f335defc92f9b20225c272862e92e32b/examples/tensorflow-models/Dockerfile.template) 
for this example starts with the published Docker images for TensorFlow ands 
the code for our TensorFlow program
  * In this example we are using the CIFAR10 example in the [TensorFlow's model zoo](https://github.com/tensorflow/models)
  * So our Dockerfile just clones that repo
  * Using TF's Docker images ensures we start with a reliable TF environment 

We need to build separate Docker images for CPU and GPU versions of TensorFlow.
  * **modes** controls whether we build images for CPU, GPU or both 
  * Our Dockerfile is a [Jinja2](http://jinja.pocoo.org/) template, so we can easily
    build docker images based on different TensorFlow versions
  
The base images controls which version of TensorFlow we will use
  * Change the base images if you want to use a different version.
  

In [12]:
# DO NOT SUBMIT
# Hack to set images to some existing ones
images = {
  "cpu": "gcr.io/cloud-ml-dev/tf-models-cpu:397ef28-dirty-aeb4db6",
  "gpu": "gcr.io/cloud-ml-dev/tf-models-gpu:591ca2e-dirty-24ef9d4",
}

In [17]:
reload(build_and_push_image)
# Set GCB project to build with GCB as opposed to locally using Docker.
# Building on GCB can be much faster because we don't have to pull/push the images to our
# local machine.
gcb_project = project
if use_gpu:
  modes = ["cpu", "gpu"]
else:
  modes = ["cpu"]

image = os.path.join(registry, "tf-models")
dockerfile = os.path.join(ROOT_DIR, "examples", "tensorflow-models", "Dockerfile.template")
base_images = {
  "cpu": "gcr.io/tensorflow/tensorflow:1.3.0",
  "gpu": "gcr.io/tensorflow/tensorflow:1.3.0-gpu",
}
images = build_and_push_image.build_and_push(dockerfile, image, modes=modes, base_images=base_images,
                                             project=gcb_project)

INFO:root:context_dir: /tmp/tmpTfJobSampleContentxtBTT7c7
INFO:root:Running gcloud container builds submit /tmp/tmpTfJobSampleContentxtBTT7c7 --tag=gcr.io/cloud-ml-dev/tf-models-cpu:84241c2-dirty-f9a532d --project=cloud-ml-dev
INFO:root:Creating temporary tarball archive of 3 file(s) totalling 1.6 KiB before compression.
INFO:root:Uploading tarball of [/tmp/tmpTfJobSampleContentxtBTT7c7] to [gs://cloud-ml-dev_cloudbuild/source/1512022271.24-8d59f068b93644ab9646fc5e25a0a48e.tgz]
INFO:root:Created [https://cloudbuild.googleapis.com/v1/projects/cloud-ml-dev/builds/016fb416-0067-4b5c-96e1-35884b6ebe0e].
INFO:root:Logs are available at [https://console.cloud.google.com/gcr/builds/016fb416-0067-4b5c-96e1-35884b6ebe0e?project=cloud-ml-dev].
INFO:root:----------------------------- REMOTE BUILD OUTPUT ------------------------------
INFO:root:starting build "016fb416-0067-4b5c-96e1-35884b6ebe0e"
INFO:root:
INFO:root:FETCHSOURCE
INFO:root:Fetching storage object: gs://cloud-ml-dev_cloudbuild/sour

## Create the Inception Datasets

We need to create the TFRecord files by running [download_and_convert_data.py](https://github.com/tensorflow/models/blob/master/research/slim/download_and_convert_data.py)
  * We submit a K8s job to run this program
  * You can skip this step if your data is already available in data_dir

In [23]:
batch_api = k8s_client.BatchV1Api(api_client)

job_name = "inception-data-" + dataset_name + "-"+ datetime.datetime.now().strftime("%y%m%d-%H%M%S")

body = {}
body['apiVersion'] = "batch/v1"
body['kind'] = "Job"
body['metadata'] = {}
body['metadata']['name'] = job_name
body['metadata']['namespace'] = namespace

# Note backoffLimit requires K8s >= 1.8
spec = """
backoffLimit: 4
template:
  spec:
    containers:
    - name: inception
      image: {image}
      workingDir: /tensorflow_models/research/slim
      command: ["python",  "download_and_convert_data.py", "--dataset_name={dataset_name}", "--dataset_dir={data_dir}"]
    restartPolicy: Never
""".format(data_dir=data_dir, dataset_name=dataset_name, image=images["cpu"])

spec_buffer = StringIO.StringIO(spec)
body['spec'] = yaml.load(spec_buffer)

try: 
    # Create a Resource
    api_response = batch_api.create_namespaced_job(namespace, body)
    print("Created job %s" % api_response.metadata.name)
except ApiException as e:
    print(
        "Exception when calling DefaultApi->apis_fqdn_v1_namespaces_namespace_resource_post: %s\n" % 
        e)


Created job inception-data-flowers-171130-062003


wait for the job to finish

In [24]:
while True:
  results = batch_api.read_namespaced_job(job_name, namespace)
  if results.status.succeeded >= 1 or results.status.failed >= 3:
    break
  print("Waiting for job %s ...." % results.metadata.name)
  time.sleep(5)

if results.status.succeeded >= 1:
  print("Job completed successfully")
else:
  print("Job failed")

Job completed successfully


## Create a TfJob

To submit a TfJob, we define a TfJob spec and then create it in our cluster

In [31]:
use_gpu = False

In [32]:
crd_api = k8s_client.CustomObjectsApi(api_client)

namespace = "default"
job_name = "inception-" + dataset_name + "-"+ datetime.datetime.now().strftime("%y%m%d-%H%M%S")
job_dir = os.path.join(job_dirs, job_name)
num_steps = 10
body = {}
body['apiVersion'] = TF_JOB_GROUP + "/" + TF_JOB_VERSION
body['kind'] = TF_JOB_KIND
body['metadata'] = {}
body['metadata']['name'] = job_name
body['metadata']['namespace'] = namespace

master_image = images["cpu"]
if use_gpu:
  master_image = images["gpu"]
spec = """
  replicaSpecs:
    - replicas: 1
      tfReplicaType: MASTER
      template:
        spec:
          containers:
            - image: {master_image}
              name: tensorflow
              workingDir: /tensorflow_models/research/slim
              command:
                - python
                - train_image_classifier.py
                - --dataset_name={dataset_name}
                - --dataset_split_name=train 
                - --dataset_dir={data_dir}
                - --model_name=inception_v3
                - --train_dir={train_dir}
          restartPolicy: OnFailure
  tfImage: {cpu_image}
  tensorBoard:
    logDir: {train_dir}
""".format(master_image=master_image, cpu_image=images["cpu"], data_dir=data_dir, 
           dataset_name=dataset_name, train_dir=job_dir)

spec_buffer = StringIO.StringIO(spec)
body['spec'] = yaml.load(spec_buffer)
if use_gpu:
  body['spec']['replicaSpecs'][0]["template"]["spec"]["containers"][0]["resources"] = {
    "limits": {
      "nvidia.com/gpu": accelerator_count,
    }    
  }

try: 
    # Create a Resource
    api_response = crd_api.create_namespaced_custom_object(TF_JOB_GROUP, TF_JOB_VERSION, namespace, TF_JOB_PLURAL, body) 
    logging.info("Created job %s", api_response["metadata"]["name"])
except ApiException as e:
    print(
        "Exception when calling DefaultApi->apis_fqdn_v1_namespaces_namespace_resource_post: %s\n" % 
        e)

INFO:root:Created job inception-flowers-171130-064345


## Monitoring your job and waiting for it to finish

We can monitor the job a number of ways
  * We can poll K8s to get the status of the TfJob
  * We can check the TensorFlow logs
      * These are available in StackDriver
  * We can access TensorBoard if the TfJob was configured to launch TensorBoard
  
Running the code below will poll K8s for the TfJob status and also print out relevant links for TensorBoard and the StackDriver logs

To access TensorBoard you will need to run **kubectl proxy** to create a proxy connection to your K8s cluster

In [114]:
# Get pod logs
v1 = k8s_client.CoreV1Api(api_client)

k8s_config.load_kube_config()
api_client = k8s_client.ApiClient()
crd_api = k8s_client.CustomObjectsApi(api_client)

master_started = False
runtime_id = None
while True:
  results = crd_api.get_namespaced_custom_object(TF_JOB_GROUP, TF_JOB_VERSION, namespace, TF_JOB_PLURAL, job_name)

  if not runtime_id:
    runtime_id = results["spec"]["RuntimeId"]
    logging.info("Job has runtime id: %s", runtime_id)
    
    tensorboard_url = "http://127.0.0.1:8001/api/v1/proxy/namespaces/{namespace}/services/tensorboard-{runtime_id}:80/".format(
    namespace=namespace, runtime_id=runtime_id)
    logging.info("Tensorboard will be available at job\n %s", tensorboard_url)

  if not master_started:
    # Get the master pod
    # TODO(jlewi): V1LabelSelector doesn't seem to help
    pods = v1.list_namespaced_pod(namespace=namespace, label_selector="runtime_id={0},job_type=MASTER".format(runtime_id))

    # TODO(jlewi): We should probably handle the case where more than 1 pod gets started.
    # TODO(jlewi): Once GKE logs pod labels we can just filter by labels to get all logs for a particular task
    # and not have to identify the actual pod.
    if pods.items:
      pod = pods.items[0]

      logging.info("master pod is %s", pod.metadata.name)
      query={
        'advancedFilter': 'resource.type="container"\nresource.labels.namespace_id="default"\nresource.labels.pod_id="{0}"'.format(pod.metadata.name), 
        'dateRangeStart': pod.metadata.creation_timestamp.isoformat(),
        'expandAll': 'false',
        'interval': 'NO_LIMIT',
        'logName': 'projects/{0}/logs/tensorflow'.format(project),
       'project': project, 
      }
      logging.info("Logs will be available in stackdriver at\n"
                   "https://console.cloud.google.com/logs/viewer?" + urllib.urlencode(query))
      master_started = True

  if results["status"]["phase"] == "Done":
    break
  print("Job status {0}".format(results["status"]["phase"]))
  time.sleep(5)
  
logging.info("Job %s", results["status"]["state"])

INFO:root:Job has runtime id: eg5k
INFO:root:Tensorboard will be available at job
 http://127.0.0.1:8001/api/v1/proxy/namespaces/default/services/tensorboard-eg5k:80/
INFO:root:master pod is master-eg5k-0-p85pk
INFO:root:Logs will be available in stackdriver at
https://console.cloud.google.com/logs/viewer?expandAll=false&dateRangeStart=2017-11-03T03%3A00%3A43%2B00%3A00&advancedFilter=resource.type%3D%22container%22%0Aresource.labels.namespace_id%3D%22default%22%0Aresource.labels.pod_id%3D%22master-eg5k-0-p85pk%22&interval=NO_LIMIT&project=cloud-ml-dev&logName=projects%2Fcloud-ml-dev%2Flogs%2Ftensorflow


Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running
Job status Running


INFO:root:Job Succeeded


## Cleanup
* Delete the GKE cluster

In [119]:
util.delete_cluster(gke, cluster_name, project, zone)

Help on function delete_cluster in module py.util:

delete_cluster(gke, name, project, zone)
    Delete the cluster.
    
    Args:
      gke: Client for GKE.
      name: Name of the cluster.
      project: Project that owns the cluster.
      zone: Zone where the cluster is running.



## Appendix

In [38]:
from kubernetes.client.models.v1_label_selector import V1LabelSelector
import urllib2
# Get pod logs
k8s_config.load_kube_config()
api_client = k8s_client.ApiClient()
v1 = k8s_client.CoreV1Api(api_client)
runtime_id = results["spec"]["RuntimeId"]
# TODO(jlewi): V1LabelSelector doesn't seem to help
pods = v1.list_namespaced_pod(namespace=namespace, label_selector="runtime_id={0},job_type=MASTER".format(runtime_id))

pod = pods.items[0]

INFO:requests.packages.urllib3.connectionpool:Starting new HTTPS connection (1): accounts.google.com


NameError: name 'results' is not defined

#### Read the Pod Logs From K8s

We can read pod logs directly from K8s and not depend on stackdriver

In [30]:
ret = v1.read_namespaced_pod_log(namespace=namespace, name=pod.metadata.name)
print(ret)

2017-10-27 23:37:29,807 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', error(104, 'Connection reset by peer'))': /api/v1/namespaces/default/pods/master-hrhh-0-wrh6g/log


INFO:tensorflow:Using config: {'_model_dir': 'gs://cloud-ml-dev_jlewi/cifar10/jobs/cifar10-171027-174224', '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_session_config': gpu_options {
  force_gpu_compatible: true
}
allow_soft_placement: true
, '_tf_random_seed': None, '_task_type': u'master', '_environment': u'cloud', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fcb715c9b10>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': '', '_log_step_count_steps': 100}
Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
INFO:tensorflow:image after unit resnet/tower_0/stage/residual_v1/: (?, 32, 32, 16)
INFO:tensorflow:image after unit resnet/tower_0/stage/residual_v1_1/: (?,

#### Fetch Logs from StackDriver Programmatically
  * On GKE pod logs are stored in stackdriver
  * These logs will stick around longer than pod logs
  * Fetching from stackNote this tends to be a little slow()

In [26]:
from google.cloud import logging as gcp_logging
pod_filter = 'resource.type="container" AND resource.labels.pod_id="master-hrhh-0-wrh6g"'
client = gcp_logging.Client(project=project)

for entry in client.list_entries(filter_=pod_filter):
  print(entry.payload.strip())

INFO:tensorflow:Using config: {'_model_dir': 'gs://cloud-ml-dev_jlewi/cifar10/jobs/cifar10-171027-174224', '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_session_config': gpu_options {
force_gpu_compatible: true
}
allow_soft_placement: true
, '_tf_random_seed': None, '_task_type': u'master', '_environment': u'cloud', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fcb715c9b10>, '_tf_config': gpu_options {
per_process_gpu_memory_fraction: 1
}
, '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': '', '_log_step_count_steps': 100}
Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
INFO:tensorflow:image after unit resnet/tower_0/stage/residual_v1/: (?, 32, 32, 16)
INFO:tensorflow:image after unit resnet/tower_0/stage/residual_v1_1/: (?, 32,